In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import streamlit as st



In [2]:
path = "../data/online_retail_II.xlsx"


# Charger toutes les feuilles sous forme de dictionnaire
dfs = pd.read_excel(path, sheet_name=None)

# Concaténer toutes les feuilles en une seule
df = pd.concat(dfs.values(), ignore_index=True)

# On met sous forme de csv pour améliorer les temps de chargement
df.to_csv("../data/online_retail_II.csv", index=False)



In [3]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,1.067371e+06,1067371,1.067371e+06,824364.000000
mean,9.938898e+00,2011-01-02 21:13:55.394028544,4.649388e+00,15324.638504
min,-8.099500e+04,2009-12-01 07:45:00,-5.359436e+04,12346.000000
25%,1.000000e+00,2010-07-09 09:46:00,1.250000e+00,13975.000000
50%,3.000000e+00,2010-12-07 15:28:00,2.100000e+00,15255.000000
75%,1.000000e+01,2011-07-22 10:23:00,4.150000e+00,16797.000000
max,8.099500e+04,2011-12-09 12:50:00,3.897000e+04,18287.000000
std,1.727058e+02,NaN,1.235531e+02,1697.464450


In [4]:
df.isnull().sum()
#We have missing values on columns "Description" and "Customer ID"

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

When Customer ID is missing, it means that it's either an anonymous transaction, a test or an error. For marleting analysis, it is usually better to get rid of these lines because we don't know the context behing this lack.

In [5]:
df = df.dropna(subset=["Customer ID"])

df.isnull().sum()



Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

We see that by deleteting lines with no Custome ID, we end up with no missing value at all, because the other missing values on Description column were also the lines with missing values on Customer ID.

Also, the dataset contains some rows with a Price of 0, which is werid because we can't classify this transaction as a sales nor as a return. It's probably an error or a free sample, and we have to delete these rows for our future marketing analysis.

In [6]:
df = df[df["Price"] != 0] #Deleting rows where "Price" = 0

Last steps of cleaning the dataset: 
Sometimes, "StockCode" is not a correct code, like "POST" (postal fees), "BANK CHARGES", or a simple letter "M" (for Manual adjustment). We also have "D" or "DOT", but it means this is an online transaction, so we'll keep them.

In [7]:
df = df[~df["StockCode"].str.contains("POST|BANK CHARGES|M", na=False)]

df = df.drop_duplicates() #We delete duplicatas

print(df)

        Invoice StockCode                          Description  Quantity  \
0        489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1        489434    79323P                   PINK CHERRY LIGHTS        12   
2        489434    79323W                  WHITE CHERRY LIGHTS        12   
3        489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4        489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
...         ...       ...                                  ...       ...   
1067365  581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
1067366  581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
1067367  581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
1067368  581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
1067369  581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

                InvoiceDate  Price  Customer ID         Country  
0       2009-12-01 07